In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
import umap

from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.cluster import KMeans


In [41]:
def expression_data():
   data = pd.read_csv('../data/data.csv')
   data = data.T
   data = data.drop(data.index[:3])
   return data

def pca(data):
    '''PCA'''
    # Select only the expression data columns for PCA
    expression_data = data

    # Standardize the data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(expression_data)

    # Apply PCA
    pca = PCA(n_components=2)  # You can change the number of components as needed
    principal_components = pca.fit_transform(scaled_data)
    return principal_components,scaled_data

def tsne(data):
    selected_columns = data
    tsne = TSNE(n_components=2, perplexity=30)  # Adjust parameters as needed

    # Perform t-SNE
    tsne_result = tsne.fit_transform(selected_columns)
    return tsne_result

def Umap(data):
    selected_columns = data  
    umap_reducer = umap.UMAP(n_components=2)  

    umap_result = umap_reducer.fit_transform(selected_columns)
    return umap_result

def ica(data):
    gene_expression = data
    ica = FastICA(n_components=2, random_state=42)


    ica.fit(gene_expression)

    independent_components = ica.transform(gene_expression)
    return independent_components

def vae(data):
    expression_data = data.values


    expression_data = (expression_data - np.min(expression_data)) / (np.max(expression_data) - np.min(expression_data))
    expression_data = tf.convert_to_tensor(expression_data, dtype=tf.float32)
    latent_dim = 2 

    encoder_inputs = keras.Input(shape=(expression_data.shape[1],))
    x = keras.layers.Dense(256, activation='relu')(encoder_inputs)
    x = keras.layers.Dense(128, activation='relu')(x)
    z_mean = keras.layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = keras.layers.Dense(latent_dim, name='z_log_var')(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = keras.layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    decoder_inputs = keras.layers.Dense(128, activation='relu')(z)
    decoder_outputs = keras.layers.Dense(expression_data.shape[1], activation='sigmoid')(decoder_inputs)

    vae = keras.Model(encoder_inputs, decoder_outputs)

    reconstruction_loss = tf.keras.losses.mean_squared_error(encoder_inputs, decoder_outputs)
    reconstruction_loss *= expression_data.shape[1]
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = tf.reduce_mean(kl_loss)
    kl_loss *= -0.5
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    vae.fit(expression_data, epochs=10, batch_size=32)

    encoder = keras.Model(encoder_inputs, z_mean)
    encoded_data = encoder.predict(expression_data)
    return encoded_data

In [45]:
#ARI for PCA
num_clusters = 2 # Change this to the number of clusters you want
pca_result,scaled_data = pca(expression_data())

# Perform K-means clustering
kmeans1 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels1 = kmeans1.fit_predict(scaled_data)  # Fit the model and obtain cluster labels

kmeans2 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels2 = kmeans2.fit_predict(scaled_data)  # Fit the model and obtain cluster labels
ari = adjusted_rand_score(cluster_labels1,cluster_labels2)
nmi = normalized_mutual_info_score(cluster_labels1,cluster_labels2)
silhouette_avg = silhouette_score(cluster_labels1.reshape(-1, 1),cluster_labels2.reshape(-1, 1))
print(f"Silhouette Score: {silhouette_avg}")
print(f"Normalized Mutual Information (NMI): {nmi}")
print(f"Adjusted Rand Index (ARI) for PCA: {ari}")


Silhouette Score: -0.9964476021314387
Normalized Mutual Information (NMI): 0.0002426712335969816
Adjusted Rand Index (ARI) for PCA: -0.0017793594306049821


d:\CS Software\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [46]:
#ARI for TSNE

num_clusters = 2 # Change this to the number of clusters you want
scaled_data = tsne(expression_data())
# Perform K-means clustering
kmeans1 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels1 = kmeans1.fit_predict(scaled_data)  # Fit the model and obtain cluster labels

kmeans2 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels2 = kmeans2.fit_predict(scaled_data)  # Fit the model and obtain cluster labels
ari = adjusted_rand_score(cluster_labels1,cluster_labels2)
nmi = normalized_mutual_info_score(cluster_labels1,cluster_labels2)
silhouette_avg = silhouette_score(cluster_labels1.reshape(-1, 1),cluster_labels2.reshape(-1, 1))
print(f"Silhouette Score: {silhouette_avg}")
print(f"Normalized Mutual Information (NMI): {nmi}")
print(f"Adjusted Rand Index (ARI) for PCA: {ari}")



d:\CS Software\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
d:\CS Software\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Silhouette Score: 1.0
Normalized Mutual Information (NMI): 1.0
Adjusted Rand Index (ARI) for PCA: 1.0


d:\CS Software\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [47]:
#ARI for UMAP

num_clusters = 2 # Change this to the number of clusters you want
scaled_data = Umap(expression_data())
# Perform K-means clustering
kmeans1 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels1 = kmeans1.fit_predict(scaled_data)  # Fit the model and obtain cluster labels

kmeans2 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels2 = kmeans2.fit_predict(scaled_data)  # Fit the model and obtain cluster labels
ari = adjusted_rand_score(cluster_labels1,cluster_labels2)
nmi = normalized_mutual_info_score(cluster_labels1,cluster_labels2)
silhouette_avg = silhouette_score(cluster_labels1.reshape(-1, 1),cluster_labels2.reshape(-1, 1))
print(f"Silhouette Score: {silhouette_avg}")
print(f"Normalized Mutual Information (NMI): {nmi}")
print(f"Adjusted Rand Index (ARI) for PCA: {ari}")

Silhouette Score: 1.0
Normalized Mutual Information (NMI): 1.0
Adjusted Rand Index (ARI) for PCA: 1.0


d:\CS Software\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
#ARI for ICA
num_clusters = 2 # Change this to the number of clusters you want
scaled_data = ica(expression_data())
# Perform K-means clustering
kmeans1 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels1 = kmeans1.fit_predict(scaled_data)  # Fit the model and obtain cluster labels

kmeans2 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels2 = kmeans2.fit_predict(scaled_data)  # Fit the model and obtain cluster labels
ari = adjusted_rand_score(cluster_labels1,cluster_labels2)
nmi = normalized_mutual_info_score(cluster_labels1,cluster_labels2)
silhouette_avg = silhouette_score(cluster_labels1.reshape(-1, 1),cluster_labels2.reshape(-1, 1))
print(f"Silhouette Score: {silhouette_avg}")
print(f"Normalized Mutual Information (NMI): {nmi}")
print(f"Adjusted Rand Index (ARI) for PCA: {ari}")

Silhouette Score: 0.9788811576254338
Normalized Mutual Information (NMI): 0.9271112837454577
Adjusted Rand Index (ARI) for PCA: 0.9669992110758683


d:\CS Software\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
#ARI for VAE
num_clusters = 2 # Change this to the number of clusters you want
scaled_data = vae(expression_data())
# Perform K-means clustering
kmeans1 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels1 = kmeans1.fit_predict(scaled_data)  # Fit the model and obtain cluster labels

kmeans2 = KMeans(n_clusters=num_clusters)  # Create a KMeans instance
cluster_labels2 = kmeans2.fit_predict(scaled_data)  # Fit the model and obtain cluster labels
ari = adjusted_rand_score(cluster_labels1,cluster_labels2)
nmi = normalized_mutual_info_score(cluster_labels1,cluster_labels2)
silhouette_avg = silhouette_score(cluster_labels1.reshape(-1, 1),cluster_labels2.reshape(-1, 1))
print(f"Silhouette Score: {silhouette_avg}")
print(f"Normalized Mutual Information (NMI): {nmi}")
print(f"Adjusted Rand Index (ARI) for PCA: {ari}")

Epoch 1/10
18/18 [==============================] - 6s 236ms/step - loss: 13202.2461
Epoch 2/10
18/18 [==============================] - 4s 227ms/step - loss: 9372.1670
Epoch 3/10
18/18 [==============================] - 4s 222ms/step - loss: 3036.6580
Epoch 4/10
18/18 [==============================] - 4s 227ms/step - loss: 394.2442
Epoch 5/10
18/18 [==============================] - 4s 228ms/step - loss: 219.3492
Epoch 6/10
18/18 [==============================] - 4s 216ms/step - loss: 147.0635
Epoch 7/10
18/18 [==============================] - 4s 221ms/step - loss: 142.7207
Epoch 8/10
18/18 [==============================] - 4s 220ms/step - loss: 125.9234
Epoch 9/10
18/18 [==============================] - 4s 223ms/step - loss: 170.0518
Epoch 10/10
18/18 [==============================] - 0s 9ms/step
Silhouette Score: 1.0
Normalized Mutual Information (NMI): 1.0
Adjusted Rand Index (ARI) for PCA: 1.0


d:\CS Software\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
